In [ ]:
import pandas as pd
import psycopg2

In [ ]:
host = 'dwl-postgres.cvwuspqh0afz.us-east-1.rds.amazonaws.com'
port = '5432'
user = 'postgres'
password = 'postgres'
db_name = 'DWL09'

In [ ]:
conn = psycopg2.connect(host, user=user,port=port,
                           passwd=password, db=dbname)

tw_list = pd.read_sql('SELECT user.screen_name, text FROM tweet13;', con=conn)

In [ ]:
tw_list.columns = ["user_account","text"]
tw_list.head()

In [ ]:
tw_list['text'] = tw_list['text'].str.lower()\
.str.replace('(@[a-z0-9]+)\w+',' ')\
.str.replace('(http\S+)', ' ')\
.str.replace('([^0-9a-z \t])',' ')\
.str.replace(' +',' ')\
.str.replace('(&gt)', '')\
.str.replace('(\xa0)', '')\
.str.replace('(&amp)', '')\
.str.replace('(&lt)', '')\
.str.replace('(<a).*(>).*(</a>)', '')\
.str.replace("(<br/>)", "")\
.str.replace("(rt)", "")

tw_list.head(n=10)

In [ ]:
tw_list.drop_duplicates(inplace=True)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

In [ ]:
#Calculating Negative, Positive, Neutral and Compound values
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
        score = SentimentIntensityAnalyzer().polarity_scores(row)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']

tw_list['neg'] = tw_list['text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['neg'])
tw_list['neu'] = tw_list['text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['neu'])
tw_list['pos'] = tw_list['text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['pos'])
tw_list['compound'] = tw_list['text'].apply(lambda x:SentimentIntensityAnalyzer().polarity_scores(x)['compound'])

tw_list.head(10)

In [ ]:
from nltk.corpus import stopwords

#nltk.download('stopwords')

additional  = ['rt','rts','retweet'] #we'll store additional stopwords here
swords = set().union(stopwords.words('english'),additional)

tw_list.drop_duplicates(subset='text',inplace=True)

tw_list['text'] = tw_list['text'].str.lower()\
.str.replace('(@[a-z0-9]+)\w+',' ')\
.str.replace('(http\S+)', ' ')\
.str.replace('([^0-9a-z \t])',' ')\
.str.replace(' +',' ')\
.str.replace('(&gt)', '')\
.str.replace('(\xa0)', '')\
.str.replace('(&amp)', '')\
.str.replace('(&lt)', '')\
.str.replace('(<a).*(>).*(</a>)', '')\
.str.replace("(<br/>)", "")\
.apply(lambda x: [i for i in x.split() if not i in swords])
#  ReviewText = ReviewText.str.replace("(<br/>)", "")
#     ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
#     ReviewText = ReviewText.str.replace('(&amp)', '')
#     ReviewText = ReviewText.str.replace('(&gt)', '')
#     ReviewText = ReviewText.str.replace('(&lt)', '')
#     ReviewText = ReviewText.str.replace('(\xa0)', ' ')  

In [ ]:
tw_list.head(n=20) 

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

tw_list['stemmed'] = tw_list['text'].apply(lambda x: [ps.stem(i) for i in x if i != ''])

In [ ]:
import nltk.sentiment.vader as vd
from nltk import download
#download('vader_lexicon')

sia = vd.SentimentIntensityAnalyzer()

In [ ]:
from nltk.tokenize import word_tokenize

#nltk.download('punkt')

tw_list['sentiment_score'] = tw_list['text'].apply(lambda x: sum([ sia.polarity_scores(i)['compound'] for i in word_tokenize( ' '.join(x) )]) )

In [ ]:
tw_list[['text','sentiment_score']].head(n=10)

In [ ]:
tw_list['sentiment_score'].apply(lambda x: round(x,)).value_counts()

In [ ]:
tw_list[tw_list['sentiment_score'] > 1]

In [ ]:
tw_list["sentiment_score'"] = None
tw_list.to_sql("tweet13", conn, if_exists="replace")